In [24]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import category_encoders as ce
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE




def load_data(file_path):
    """Load data from a CSV file into a pandas DataFrame."""
    return pd.read_csv(file_path)

In [25]:
df = load_data('data/bank_data_train.csv')
TRESHOLD = 0.7
print(df.shape)

(355190, 116)


In [26]:
def explore_data(df):
    """Display basic information about the DataFrame and drop 'ID' if present. Returns the (possibly) modified DataFrame."""
    print("DataFrame Info:")
    df.info()
    
    # drop unnecessary column if it exists
    if 'ID' in df.columns:
        df = df.drop(columns=['ID'])
        print("\nDropped column: 'ID'")
    else:
        print("\nColumn 'ID' not found; skipping drop.")
    
    print("\nDataFrame Description (numeric):")
    display(df.describe())
    
    print("\nDataFrame Description (object):")
    display(df.describe(include=['object']))
    
    # optionally show missing values and plot in grid format
    print("\nMissing Values:")
    df_info = pd.DataFrame(df.isnull().sum(), columns=['Missing Values'])
    df_info['Percentage'] = (df_info['Missing Values'] / len(df)) * 100

    # fig, ax = plt.subplots(figsize=(15, 10))
    # df_info['Percentage'].plot(kind='bar', ax=ax, color='tab:blue')
    # ax.set_title('Missing Values in Each Column')
    # ax.set_ylabel('Percentage (%)')
    # ax.set_xlabel('Columns')
    # plt.xticks(rotation=45, ha='right')
    # plt.tight_layout()
    # plt.show()
    
    
    #siwich column into rows for better display

    df_info = df_info.transpose()
    display(df_info)
    # display(df.isnull().sum())
    
    # check the distribution of target column
    print(df['TARGET'].value_counts())
    return df

df = explore_data(df)

DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 355190 entries, 0 to 355189
Columns: 116 entries, ID to TARGET
dtypes: float64(94), int64(9), object(13)
memory usage: 314.3+ MB

Dropped column: 'ID'

DataFrame Description (numeric):


,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,...,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
count,355190.000000,316867.000000,155163.0,60550.000000,155163.0,155163.0,355190.000000,77112.000000,77112.000000,316867.000000,...,355190.000000,95713.000000,93448.000000,98175.000000,95713.000000,355190.000000,93448.000000,93448.000000,93448.000000,355190.000000
mean,0.105225,0.044045,0.0,50.947498,0.0,0.0,0.001305,0.416896,0.414572,0.085249,...,0.007309,0.055074,0.025707,0.049943,0.009252,0.004309,0.013938,0.013938,0.013938,0.081435
std,0.431372,0.108449,0.0,21.777855,0.0,0.0,0.029118,0.316493,0.338612,0.142310,...,0.066681,0.215909,0.115732,0.185830,0.092789,0.059852,0.097099,0.097099,0.097099,0.273503
min,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.006944,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.0,33.000000,0.0,0.0,0.000000,0.166667,0.139645,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,0.000000,0.0,54.000000,0.0,0.0,0.000000,0.300000,0.285714,0.027117,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,0.036608,0.0,72.000000,0.0,0.0,0.000000,0.571429,0.661195,0.110005,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,11.000000,1.000000,0.0,89.000000,0.0,0.0,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000



DataFrame Description (object):


,CLNT_TRUST_RELATION,APP_MARITAL_STATUS,APP_KIND_OF_PROP_HABITATION,CLNT_JOB_POSITION_TYPE,CLNT_JOB_POSITION,APP_DRIVING_LICENSE,APP_EDUCATION,APP_TRAVEL_PASS,APP_CAR,APP_POSITION_TYPE,APP_EMP_TYPE,APP_COMP_TYPE,PACK
count,69421,68234,59361,44781,210811,57257,68104,57257,57256,60545,67362,67362,355190
unique,21,13,5,4,19588,2,17,2,2,4,4,4,12
top,FRIEND,M,SO,SPECIALIST,ДИРЕКТОР,N,H,N,N,SPECIALIST,PRIVATE,PRIVATE,102
freq,24896,30724,28056,25123,11200,36332,42459,52750,32843,36622,59087,59087,116986



Missing Values:


,CR_PROD_CNT_IL,AMOUNT_RUB_CLO_PRC,PRC_ACCEPTS_A_EMAIL_LINK,APP_REGISTR_RGN_CODE,PRC_ACCEPTS_A_POS,PRC_ACCEPTS_A_TK,TURNOVER_DYNAMIC_IL_1M,CNT_TRAN_AUT_TENDENCY1M,SUM_TRAN_AUT_TENDENCY1M,AMOUNT_RUB_SUP_PRC,...,REST_DYNAMIC_CC_3M,MED_DEBT_PRC_YWZ,LDEAL_ACT_DAYS_PCT_TR3,LDEAL_ACT_DAYS_PCT_AAVG,LDEAL_DELINQ_PER_MAXYWZ,TURNOVER_DYNAMIC_CC_3M,LDEAL_ACT_DAYS_PCT_TR,LDEAL_ACT_DAYS_PCT_TR4,LDEAL_ACT_DAYS_PCT_CURR,TARGET
Missing Values,0.0,38323.000000,200027.000000,294640.000000,200027.000000,200027.000000,0.0,278078.000000,278078.000000,38323.000000,...,0.0,259477.000000,261742.000000,257015.000000,259477.000000,0.0,261742.000000,261742.000000,261742.000000,0.0
Percentage,0.0,10.789437,56.315493,82.952786,56.315493,56.315493,0.0,78.289929,78.289929,10.789437,...,0.0,73.053014,73.690701,72.359864,73.053014,0.0,73.690701,73.690701,73.690701,0.0


TARGET
0    326265
1     28925
Name: count, dtype: int64


In [27]:
326265/df.shape[0]

0.918564711844365

In [28]:
from scipy.stats import chi2_contingency
import numpy as np

def cramers_v(x, y):
    """Measure association between categorical x and binary y (TARGET)"""
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2 / n
    r, k = confusion_matrix.shape
    return np.sqrt(phi2 / min(k-1, r-1))

In [29]:
# Annalyze data whitch to keep or drop
print("\nAnalyzing columns with >70% missing values for correlation with TARGET...")
print(f"df shape before dropping columns: {df.shape}")
high_missing = df.columns[df.isnull().sum() / len(df) > TRESHOLD]
for col in high_missing:
    if col == 'TARGET':
        continue
    # coerce to numeric where possible, compute correlation with TARGET
    
    if df[col].dtype == 'object':
        # categorical column
        corr = cramers_v(df[col].dropna(), df.loc[df[col].notnull(), 'TARGET'])
        if corr < 0.1:
            print(f"Dropping column {col} due to low correlation ({corr:.3f}) with TARGET")
            df = df.drop(columns=[col])
    else:
        # numerical column
        series_num = pd.to_numeric(df[col], errors='coerce')
        corr = series_num.corr(df['TARGET'])
        if pd.notnull(corr) and abs(corr) < 0.05:
            print(f"Dropping column {col} due to low correlation ({corr:.3f}) with TARGET")
            df = df.drop(columns=[col])
    
    
    # series_num = pd.to_numeric(df[col], errors='coerce')
    
    
    
    # corr = cramers_v(df[col].dropna(), df.loc[df[col].notnull(), 'TARGET'])
    # missing_pct = df[col].isnull().sum() / len(df) * 100
    # corr_str = f"{corr:.3f}" if pd.notnull(corr) else "N/A"
    # print(f"{col}: Missing={missing_pct:.1f}%, Correlation={corr_str}")
print(f"df shape after dropping columns: {df.shape}")


Analyzing columns with >70% missing values for correlation with TARGET...
df shape before dropping columns: (355190, 115)
Dropping column APP_REGISTR_RGN_CODE due to low correlation (-0.028) with TARGET
Dropping column CLNT_TRUST_RELATION due to low correlation (0.029) with TARGET
Dropping column APP_MARITAL_STATUS due to low correlation (0.030) with TARGET
Dropping column APP_KIND_OF_PROP_HABITATION due to low correlation (0.008) with TARGET
Dropping column CLNT_JOB_POSITION_TYPE due to low correlation (0.036) with TARGET
Dropping column APP_DRIVING_LICENSE due to low correlation (0.031) with TARGET
Dropping column APP_EDUCATION due to low correlation (0.058) with TARGET
Dropping column APP_TRAVEL_PASS due to low correlation (0.024) with TARGET
Dropping column APP_CAR due to low correlation (0.028) with TARGET
Dropping column APP_POSITION_TYPE due to low correlation (0.034) with TARGET
Dropping column APP_EMP_TYPE due to low correlation (0.030) with TARGET
Dropping column DEAL_YQZ_IR

In [30]:
for col in df:
    column_type = df[col].dtype
    if column_type != 'int64' and column_type != 'float64':
        print(f"{col}: {column_type}")

CLNT_JOB_POSITION: object
PACK: object


In [31]:
import numpy as np


def normalize_categorical_columns(df):
    """
    Normalize categorical columns by converting to lowercase/uppercase
    and stripping whitespace
    """
    cat_cols = df.select_dtypes(include=['object']).columns.tolist()
    
    for col in cat_cols:
        # Convert to string, lowercase, and strip whitespace
        df[col] = df[col].astype(str).str.upper()
        df[col] = df[col].str.strip()
        df[col] = df[col].replace('NAN', np.nan)  # Replace 'NA' strings with NaN
        df[col] = df[col].replace('', np.nan)  # Replace empty strings with NaN
    
    return df
df = normalize_categorical_columns(df)

In [32]:
for col in df:
    column_type = df[col].dtype
    if column_type != 'int64' and column_type != 'float64':
        print(f"{col}: {column_type}")


# Count non-null occurrences of each job position
existing_values = {}
for val in df['PACK']:
    if pd.isnull(val):
        continue
    existing_values[val] = existing_values.get(val, 0) + 1
corr = cramers_v(df['PACK'].dropna(), df.loc[df['PACK'].notnull(), 'TARGET'])
missing_pct = df['PACK'].isnull().sum() / len(df) * 100
print(f"PACK: Missing={missing_pct:.1f}%, Correlation={corr:.3f}")
print(f"existing_values length: {len(existing_values)}")
existing_values

CLNT_JOB_POSITION: object
PACK: object
PACK: Missing=0.0%, Correlation=0.071
existing_values length: 12


{'K01': 77083,
 '102': 116986,
 '105': 44936,
 'O01': 50478,
 '103': 24860,
 '101': 1816,
 '107': 27952,
 '301': 4208,
 '104': 6776,
 '108': 2,
 '109': 86,
 'M01': 7}

In [33]:
def fit_encoders(X_train, y_train):
    """
    Fit encoders on training data and return fitted encoders.
    """
    # Get object columns
    obj_cols = X_train.select_dtypes(include=['object']).columns.tolist()
    
    # Try numeric conversion
    numeric_conversions = {}
    for col in obj_cols:
        converted = pd.to_numeric(X_train[col], errors='coerce')
        if converted.notna().mean() > 0.9:
            numeric_conversions[col] = True
    print("numeric conversion", numeric_conversions)
    # Update object columns
    obj_cols = [col for col in obj_cols if col not in numeric_conversions]
    
    # Categorize by cardinality
    low_card_cols = []
    high_card_cols = []
    
    for col in obj_cols:
        if X_train[col].isnull().all():
            continue
        
        unique_count = X_train[col].nunique(dropna=True)
        
        if unique_count < 10:
            low_card_cols.append(col)
        else:
            high_card_cols.append(col)
    
    # Fit target encoder on training data
    target_encoder = None
    if high_card_cols:
        target_encoder = ce.TargetEncoder(cols=high_card_cols, smoothing=1.0)
        valid_mask = X_train[high_card_cols].notna().all(axis=1) & y_train['TARGET'].notna()
        target_encoder.fit(X_train.loc[valid_mask, high_card_cols], y_train.loc[valid_mask, 'TARGET'])
    
    return {
        'numeric_conversions': numeric_conversions,
        'low_card_cols': low_card_cols,
        'high_card_cols': high_card_cols,
        'target_encoder': target_encoder,
        'global_mean': y_train['TARGET'].mean()
    }

In [34]:

def transform_with_encoders(X, encoders_dict):
    """
    Transform data using pre-fitted encoders.
    """
    X = X.copy()
    
    # Apply numeric conversions
    for col in encoders_dict['numeric_conversions']:
        X[col] = pd.to_numeric(X[col], errors='coerce')
    
    # Drop all-null columns
    obj_cols = X.select_dtypes(include=['object']).columns
    for col in obj_cols:
        if X[col].isnull().all():
            X = X.drop(columns=[col])
    
    # Target encode high-cardinality columns
    if encoders_dict['high_card_cols'] and encoders_dict['target_encoder']:
        high_card_cols = encoders_dict['high_card_cols']
        print(f"\n\nTarget encoding: {high_card_cols}\n\n")
        for col in high_card_cols:
            if col in X.columns:
                # Transform using fitted encoder
                print(f"Encoding column: {col}")
                print()
                try:
                    # Transform using the encoder fitted on all high-cardinality cols,
                    # then take the encoded series for the current column
                    encoded = encoders_dict['target_encoder'].transform(X[encoders_dict['high_card_cols']])[col]
                    X[col] = encoded
                    # Fill unseen / missing encodings with global mean
                    X[col].fillna(encoders_dict['global_mean'], inplace=True)
                except Exception:
                    # If transform fails, fall back to global mean
                    X[col] = encoders_dict['global_mean']
        #         X = X.drop(columns=[col])
                
                
                
        #         X[f"{col}"] = encoded[col]
                
        #         # Fill nulls with global mean
        #         X[f"{col}"].fillna(encoders_dict['global_mean'], inplace=True)
        
        # Drop original columns
        # X = X.drop(columns=[col for col in high_card_cols if col in X.columns])
    
    # One-hot encode low-cardinality columns
    if encoders_dict['low_card_cols']:
        print(f"\nOne-hot encoding: {encoders_dict['low_card_cols']}")

        X = pd.get_dummies(X, columns=encoders_dict['low_card_cols'], drop_first=True)
    
    return X

In [35]:
# split data train test


y = pd.DataFrame()
y['TARGET'] = df['TARGET']
X = df.drop(columns=['TARGET'])
X_train, X_test, y_train, y_test = train_test_split(
                            X,y,
                            test_size=0.2,
                            random_state=42,
                            stratify=y)

print(f"Train shape: {X_train.shape}, Test shape: {X_test.shape}")


Train shape: (284152, 87), Test shape: (71038, 87)


In [36]:
# Fit encoders on TRAINING data only
# print("\n" + "="*80)
print("Fitting encoders on training data...")
print("="*80)
encoders = fit_encoders(X_train, y_train)
for key, value in encoders.items():
        print(f"{key}: {value}")

Fitting encoders on training data...
numeric conversion {}
numeric_conversions: {}
low_card_cols: []
high_card_cols: ['CLNT_JOB_POSITION', 'PACK']
target_encoder: TargetEncoder(cols=['CLNT_JOB_POSITION', 'PACK'], smoothing=1.0)
global_mean: 0.08143528815563501


In [37]:
print("\nTransforming training data...")
X_train_encoded = transform_with_encoders(X_train, encoders)

print("\nTransforming test data...")
X_test_encoded = transform_with_encoders(X_test, encoders)


print(f"\nAfter encoding - Train: {X_train_encoded.shape}, Test: {X_test_encoded.shape}")


Transforming training data...


Target encoding: ['CLNT_JOB_POSITION', 'PACK']


Encoding column: CLNT_JOB_POSITION

Encoding column: PACK


Transforming test data...


Target encoding: ['CLNT_JOB_POSITION', 'PACK']


Encoding column: CLNT_JOB_POSITION

Encoding column: PACK


After encoding - Train: (284152, 87), Test: (71038, 87)


/tmp/ipykernel_6251/204901964.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X[col].fillna(encoders_dict['global_mean'], inplace=True)
/tmp/ipykernel_6251/204901964.py:32: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=Tru

In [38]:
# Just to make sure not needed in our case -------------------------------------
# Align columns (ensure train and test have same columns)
print("\nAligning train/test columns...")
train_cols = set(X_train_encoded.columns)
test_cols = set(X_test_encoded.columns)

# Add missing columns to test (fill with 0)
for col in train_cols - test_cols:
    X_test_encoded[col] = 0
    print(f"Added missing column to test: {col}")

# Remove extra columns from test
for col in test_cols - train_cols:
    X_test_encoded = X_test_encoded.drop(columns=[col])
    print(f"Removed extra column from test: {col}")
#-------------------------------------------------------------------------------------


Aligning train/test columns...


In [39]:
# Reorder test columns to match train

#! Mandatory step to ensure columns are in the same order after encoding

X_test_encoded = X_test_encoded[X_train_encoded.columns]

In [40]:
print(f"\nTrain nulls: {X_train_encoded.isnull().sum().sum()}")
print(f"Test nulls: {X_test_encoded.isnull().sum().sum()}")


Train nulls: 8813805
Test nulls: 2202381


In [41]:
imputation_values = X_train_encoded.mean()

for col in X_train_encoded.columns:
    if X_train_encoded[col].isnull().any():
        X_train_encoded[col].fillna(imputation_values[col], inplace=True)
    
    if X_test_encoded[col].isnull().any():
        X_test_encoded[col].fillna(imputation_values[col], inplace=True)
print(f"\nAfter fillna with mean\nTrain nulls: {X_train_encoded.isnull().sum().sum()}")
print(f"Test nulls: {X_test_encoded.isnull().sum().sum()}")


After fillna with mean
Train nulls: 0
Test nulls: 0


/tmp/ipykernel_6251/4282412580.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X_train_encoded[col].fillna(imputation_values[col], inplace=True)
/tmp/ipykernel_6251/4282412580.py:8: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inp

In [42]:
# Balence Dataset for training set only

print("Balancing training set using SMOTE...")
print(f"Before SMOTE: {y_train['TARGET'].value_counts()} | X shape: {X_train_encoded.shape}")
smote = SMOTE(sampling_strategy='minority', random_state=42)
X_train_balanced, y_train_balanced = smote.fit_resample(X_train_encoded, y_train)
print(f"After SMOTE: {y_train_balanced['TARGET'].value_counts()} | X shape: {X_train_balanced.shape}")

Balancing training set using SMOTE...
Before SMOTE: TARGET
0    261012
1     23140
Name: count, dtype: int64 | X shape: (284152, 87)
After SMOTE: TARGET
0    261012
1    261012
Name: count, dtype: int64 | X shape: (522024, 87)
